In [157]:
# Extensions
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

In [1]:
# Imports
import torch
import pandas as pd
import numpy as np

In [2]:
# Configuration
ID = torch.Tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
x = torch.randn(10, 4, 100)
x_rev_comp = torch.randn(10, 4, 100)
y = torch.randint(0, 2, (10, 1))
batch = (ID, x, x_rev_comp, y)

In [3]:
# Helper functions
def move_module_to_cpu(module):
    module.to("cpu")
    for child in module.children():
        move_module_to_cpu(child)

# Basic models

## FCN

In [25]:
from eugene.models.zoo import FCN

In [33]:
model = FCN(
    input_len=100,
    output_dim=10,
    dense_kwargs={
        "hidden_dims": [50, 25],
        "activations": "relu",
        "batchnorm": True,
    }
)
model_out = model(x)
model, model_out.shape

(FCN(
   (dense_block): DenseBlock(
     (layers): Sequential(
       (0): Linear(in_features=400, out_features=50, bias=True)
       (1): ReLU()
       (2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (3): Linear(in_features=50, out_features=25, bias=True)
       (4): ReLU()
       (5): BatchNorm1d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (6): Linear(in_features=25, out_features=10, bias=True)
     )
   )
 ),
 torch.Size([10, 10]))

In [35]:
model = FCN(
    input_len=100,
    output_dim=1,
    dense_kwargs={
        "hidden_dims": [100, 50, 25],
        "activations": ["relu", None, None],
        "batchnorm": True,
        "batchnorm_first": True,
        "biases": False
    }
)
model_out = model(x)
model, model_out.shape

(FCN(
   (dense_block): DenseBlock(
     (layers): Sequential(
       (0): Linear(in_features=400, out_features=100, bias=False)
       (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU()
       (3): Linear(in_features=100, out_features=50, bias=False)
       (4): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (5): Linear(in_features=50, out_features=25, bias=False)
       (6): BatchNorm1d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (7): Linear(in_features=25, out_features=1, bias=False)
     )
   )
 ),
 torch.Size([10, 1]))

In [39]:
model = FCN(
    input_len=100,
    output_dim=1,
    dense_kwargs=dict(hidden_dims=[100, 50, 25], activations=["relu", None, None], dropout_rates=[0.1, 0.5], batchnorm=True, batchnorm_first=True, biases=False),
)
model_out = model(x)
model, model_out.shape

(FCN(
   (dense_block): DenseBlock(
     (layers): Sequential(
       (0): Linear(in_features=400, out_features=100, bias=False)
       (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU()
       (3): Dropout(p=0.1, inplace=False)
       (4): Linear(in_features=100, out_features=50, bias=False)
       (5): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (6): Dropout(p=0.5, inplace=False)
       (7): Linear(in_features=50, out_features=25, bias=False)
       (8): BatchNorm1d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (9): Linear(in_features=25, out_features=1, bias=False)
     )
   )
 ),
 torch.Size([10, 1]))

In [38]:
model = FCN(
    input_len=100,
    output_dim=1,
    dense_kwargs=dict(hidden_dims=[100, 50, 25], activations=["relu", None, None], dropout_rates=0.1, batchnorm=True, batchnorm_first=True, biases=False),
)
model_out = model(x)
model, model_out.shape

(FCN(
   (dense_block): DenseBlock(
     (layers): Sequential(
       (0): Linear(in_features=400, out_features=100, bias=False)
       (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU()
       (3): Dropout(p=0.1, inplace=False)
       (4): Linear(in_features=100, out_features=50, bias=False)
       (5): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (6): Dropout(p=0.1, inplace=False)
       (7): Linear(in_features=50, out_features=25, bias=False)
       (8): BatchNorm1d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (9): Dropout(p=0.1, inplace=False)
       (10): Linear(in_features=25, out_features=1, bias=False)
     )
   )
 ),
 torch.Size([10, 1]))

## CNN

In [40]:
from eugene.models.zoo import CNN

In [44]:
model = CNN(
    input_len=100,
    output_dim=10,
    conv_kwargs={
        "input_channels": 4,
        "conv_channels": [10, 10],
        "conv_kernels": [5, 3],
        "activations": [],
        "pool_types": []
    }
)
model_out = model(x)
model, model_out.shape

(CNN(
   (conv1d_tower): Conv1DTower(
     (layers): Sequential(
       (0): Conv1d(4, 10, kernel_size=(5,), stride=(1,), padding=valid)
       (1): Conv1d(10, 10, kernel_size=(3,), stride=(1,), padding=valid)
     )
   )
   (dense_block): DenseBlock(
     (layers): Sequential(
       (0): Linear(in_features=940, out_features=10, bias=True)
     )
   )
 ),
 torch.Size([10, 10]))

In [51]:
model = CNN(
    input_len=100,
    output_dim=10,
    conv_kwargs={
        "input_channels": 4,
        "conv_channels": [10, 10],
        "conv_kernels": [5, 3],
        "activations": ["sigmoid", "relu"],
        "pool_types": ["avg", "max"]
    },
    dense_kwargs={
        "hidden_dims": [50, 25],
        "activations": "relu",
        "batchnorm": True,
    }
)
model_out = model(x)
model, model_out.shape

(CNN(
   (conv1d_tower): Conv1DTower(
     (layers): Sequential(
       (0): Conv1d(4, 10, kernel_size=(5,), stride=(1,), padding=valid)
       (1): Sigmoid()
       (2): AvgPool1d(kernel_size=1, stride=1, padding=(0,))
       (3): Conv1d(10, 10, kernel_size=(3,), stride=(1,), padding=valid)
       (4): ReLU()
       (5): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
     )
   )
   (dense_block): DenseBlock(
     (layers): Sequential(
       (0): Linear(in_features=940, out_features=50, bias=True)
       (1): ReLU()
       (2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (3): Linear(in_features=50, out_features=25, bias=True)
       (4): ReLU()
       (5): BatchNorm1d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (6): Linear(in_features=25, out_features=10, bias=True)
     )
   )
 ),
 torch.Size([10, 10]))

In [54]:
model = CNN(
    input_len=100,
    output_dim=10,
    conv_kwargs=dict(input_channels=4, conv_channels=[10, 10], conv_kernels=[5, 3], activations=["relu", "relu"], pool_types=["avg", "max"]),
    dense_kwargs=dict(hidden_dims=[50, 25], activations="relu", batchnorm=True, dropout_rates=0.1),
)
model_out = model(x)
model, model_out.shape

(CNN(
   (conv1d_tower): Conv1DTower(
     (layers): Sequential(
       (0): Conv1d(4, 10, kernel_size=(5,), stride=(1,), padding=valid)
       (1): ReLU()
       (2): AvgPool1d(kernel_size=1, stride=1, padding=(0,))
       (3): Conv1d(10, 10, kernel_size=(3,), stride=(1,), padding=valid)
       (4): ReLU()
       (5): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
     )
   )
   (dense_block): DenseBlock(
     (layers): Sequential(
       (0): Linear(in_features=940, out_features=50, bias=True)
       (1): ReLU()
       (2): Dropout(p=0.1, inplace=False)
       (3): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (4): Linear(in_features=50, out_features=25, bias=True)
       (5): ReLU()
       (6): Dropout(p=0.1, inplace=False)
       (7): BatchNorm1d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (8): Linear(in_features=25, out_features=10, bias=True)
     )
   )
 ),
 torch.Size([10, 10]))

## RNN

In [55]:
from eugene.models.zoo import RNN

In [57]:
model = RNN(
    input_len=100,
    output_dim=10,
    recurrent_kwargs={
        "hidden_dim": 10,
        "num_layers": 2,
        "bidirectional": True
    }
)
model_out = model(x.reshape(10, 100, 4))
model, model_out.shape

(RNN(
   (recurrent_block): RecurrentBlock(
     (layers): LSTM(4, 10, num_layers=2, batch_first=True, bidirectional=True)
   )
   (dense_block): DenseBlock(
     (layers): Sequential(
       (0): Linear(in_features=20, out_features=10, bias=True)
     )
   )
 ),
 torch.Size([10, 10]))

In [62]:
model = RNN(
    input_len=100,
    output_dim=10,
    recurrent_kwargs=dict(hidden_dim=10, num_layers=2, dropout_rates=0.1, bidirectional=True, unit_type="lstm"),
)
model_out = model(x.reshape(10, 100, 4))
model, model_out.shape

(RNN(
   (recurrent_block): RecurrentBlock(
     (layers): LSTM(4, 10, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
   )
   (dense_block): DenseBlock(
     (layers): Sequential(
       (0): Linear(in_features=20, out_features=10, bias=True)
     )
   )
 ),
 torch.Size([10, 10]))

In [63]:
model = RNN(
    input_len=100,
    output_dim=10,
    recurrent_kwargs=dict(hidden_dim=10, num_layers=2, dropout_rates=0.1, bidirectional=False, unit_type="rnn"),
)
model_out = model(x.reshape(10, 100, 4))
model, model_out.shape

(RNN(
   (recurrent_block): RecurrentBlock(
     (layers): RNN(4, 10, num_layers=2, batch_first=True, dropout=0.1)
   )
   (dense_block): DenseBlock(
     (layers): Sequential(
       (0): Linear(in_features=10, out_features=10, bias=True)
     )
   )
 ),
 torch.Size([10, 10]))

In [64]:
model = RNN(
    input_len=100,
    output_dim=10,
    recurrent_kwargs=dict(hidden_dim=128, num_layers=2, dropout_rates=0.3, bidirectional=False, unit_type="gru"),
)
model_out = model(x.reshape(10, 100, 4))
model, model_out.shape

(RNN(
   (recurrent_block): RecurrentBlock(
     (layers): GRU(4, 128, num_layers=2, batch_first=True, dropout=0.3)
   )
   (dense_block): DenseBlock(
     (layers): Sequential(
       (0): Linear(in_features=128, out_features=10, bias=True)
     )
   )
 ),
 torch.Size([10, 10]))

## Hybrid

In [65]:
from eugene.models.zoo import Hybrid

In [67]:
model = Hybrid(
    input_len=100,
    output_dim=10,
    conv_kwargs={
        "input_channels": 4,
        "conv_channels": [10, 10],
        "conv_kernels": [5, 3],
        "activations": "relu",
        "pool_types": "max"
    },
    recurrent_kwargs={
        "hidden_dim": 10,
        "num_layers": 10,
        "bidirectional": True
    }
)
model_out = model(x)
model, model_out.shape

(Hybrid(
   (conv1d_tower): Conv1DTower(
     (layers): Sequential(
       (0): Conv1d(4, 10, kernel_size=(5,), stride=(1,), padding=valid)
       (1): ReLU()
       (2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
       (3): Conv1d(10, 10, kernel_size=(3,), stride=(1,), padding=valid)
       (4): ReLU()
       (5): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
     )
   )
   (recurrent_block): RecurrentBlock(
     (layers): LSTM(10, 10, num_layers=10, batch_first=True, bidirectional=True)
   )
   (dense_block): DenseBlock(
     (layers): Sequential(
       (0): Linear(in_features=20, out_features=10, bias=True)
     )
   )
 ),
 torch.Size([10, 10]))

In [68]:
model = Hybrid(
    input_len=100,
    output_dim=10,
    conv_kwargs=dict(input_channels=4, conv_channels=[10, 10], conv_kernels=[5, 3], activations="relu", pool_types="max"),
    recurrent_kwargs=dict(hidden_dim=10, num_layers=10, dropout_rates=0.1, bidirectional=True, unit_type="lstm"),
    dense_kwargs=dict(hidden_dims=[50, 25], activations="relu", batchnorm=True, dropout_rates=0.1),
)
model_out = model(x)
model, model_out.shape

(Hybrid(
   (conv1d_tower): Conv1DTower(
     (layers): Sequential(
       (0): Conv1d(4, 10, kernel_size=(5,), stride=(1,), padding=valid)
       (1): ReLU()
       (2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
       (3): Conv1d(10, 10, kernel_size=(3,), stride=(1,), padding=valid)
       (4): ReLU()
       (5): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
     )
   )
   (recurrent_block): RecurrentBlock(
     (layers): LSTM(10, 10, num_layers=10, batch_first=True, dropout=0.1, bidirectional=True)
   )
   (dense_block): DenseBlock(
     (layers): Sequential(
       (0): Linear(in_features=20, out_features=50, bias=True)
       (1): ReLU()
       (2): Dropout(p=0.1, inplace=False)
       (3): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (4): Linear(in_features=50, out_features=25, bias=True)
       (5): ReLU()
       (6): Dropout(p=0.1, inplace=False)
       (7): BatchNorm1d(25, e

## TutorialCNN

In [4]:
from eugene.models.zoo import TutorialCNN

In [6]:
model = TutorialCNN(
    input_len=100,
    output_dim=10
)
model_out = model(x)
model, model_out.shape

(TutorialCNN(
   (conv1): Conv1d(4, 30, kernel_size=(21,), stride=(1,))
   (dense): Linear(in_features=30, out_features=10, bias=True)
 ),
 torch.Size([10, 10]))

# DONE

---

# Scratch